In [7]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [8]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-11-26 21:02:55--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M  64.3MB/s    in 0.4s    

2025-11-26 21:02:56 (64.3 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [9]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [10]:
# 1. Filter data
# Calculate counts on the ORIGINAL data to preserve more books
user_counts = df_ratings['user'].value_counts()
active_users = user_counts[user_counts >= 200].index

book_counts = df_ratings['isbn'].value_counts()
popular_books = book_counts[book_counts >= 100].index

# Apply both filters simultaneously
df_ratings_filtered = df_ratings[
    (df_ratings['user'].isin(active_users)) &
    (df_ratings['isbn'].isin(popular_books))
]

# 2. Merge with book titles
df_final = df_ratings_filtered.merge(df_books, on='isbn')

# 3. Create Pivot Table & Matrix
# Use 'title' for the index to aggregate ratings for the same book
pt = df_final.pivot_table(index='title', columns='user', values='rating').fillna(0)
book_matrix = csr_matrix(pt.values)

# 4. Train Model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(book_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [11]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  # Check if book exists
  if book not in pt.index:
    print(f"Book '{book}' not found.")
    return [book, []]

  # Find neighbors
  book_iloc = pt.index.get_loc(book)
  distances, indices = model_knn.kneighbors(pt.iloc[book_iloc].values.reshape(1, -1), n_neighbors=6)

  # Format results (reverse order for the test)
  recommended_books = []
  for i in range(1, len(distances.flatten())):
    recommended_books.append([pt.index[indices.flatten()[i]], distances.flatten()[i]])

  recommended_books = sorted(recommended_books, key=lambda x: x[1], reverse=True)

  return [book, recommended_books]

In [12]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", np.float32(0.8016211)], ['The Weight of Water', np.float32(0.77085835)], ['The Surgeon', np.float32(0.7699411)], ['I Know This Much Is True', np.float32(0.7677075)], ['The Lovely Bones: A Novel', np.float32(0.7234864)]]]
You passed the challenge! 🎉🎉🎉🎉🎉
